In [ ]:
%load_ext autoreload
%autoreload 2


In [4]:

from buttermilk import BM
from promptflow.tracing import trace, start_trace
from rich import print as rprint

import os

from hydra import initialize, compose
from omegaconf import OmegaConf

# Load config, specifying overrides for our particular job
with initialize(version_base=None, config_path="./conf"):
    cfg = compose(config_name='config',
                  overrides=["+data=drag", "+step=ordinary"])

# Load the main ButterMilk singleton instance
# This takes care of credentials, save paths, and other defaults
bm = BM(cfg=cfg)

# Print config
rprint(OmegaConf.to_container(bm.cfg, resolve=True))



2024-11-04 17:31:10 J5HW6L4KT6 buttermilk buttermilk.py[ 182] DEBUG Config passed in but we already have one loaded. Overwriting.


{
    'name': 'automod',
    'job': 'judge',
    'secret_provider': 'azure',
    'logger': 'gcp',
    'verbose': True,
    'models_secret': 'models',
    'save_dest': 'gcp',
    'save_dir': None,
    'gcp': {'project': 'dmrc-analysis', 'region': 'us-central1', 'bucket': 'dmrc-analysis'},
    'azure': {'vault': 'https://suzorvault.vault.azure.net/', 'resource_group': 'rg-suzor_ai'},
    'aws': {'secret_name': 'secret', 'region': 'us-east-1'},
    'data': [
        {
            'drag': None,
            'name': 'drag queens - alt text',
            'type': 'file',
            'uri': 'gs://dmrc-platforms/data/drag_train.jsonl',
            'columns': {'record_id': 'id', 'content': 'alt_text', 'groundtruth': 'expected'}
        }
    ],
    'step': [
        {
            'name': 'judger',
            'num_runs': 1,
            'concurrent': 20,
            'agent': {
                'name': 'lc_judge',
                'template': 'judge',
                'criteria': 'criteria_ordinary',
                'formatting': 'json_rules',
                'model': ['haiku']
            },
            'parameters': None
        }
    ],
    'run': {'platform': 'local'}
}

In [5]:
# Create an orchestrator to conduct all combinations of jobs we want to run
from buttermilk.runner.orchestrator import MultiFlowOrchestrator
orchestrator = MultiFlowOrchestrator(step=cfg.step[0], data=cfg.data)


In [3]:
await orchestrator.run_tasks()


  + Exception Group Traceback (most recent call last):
  |   File "/opt/homebrew/Caskroom/miniconda/base/envs/bm/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3575, in run_code
  |     await eval(code_obj, self.user_global_ns, self.user_ns)
  |   File "/var/folders/m0/dnlbtw0563gbgn18y50l5vjm0000gq/T/ipykernel_30916/1626473268.py", line 1, in <module>
  |     await orchestrator.run_tasks()
  |   File "/Users/suzor/src/buttermilk/buttermilk/runner/orchestrator.py", line 86, in run_tasks
  |     async with asyncio.TaskGroup() as tg:
  |   File "/opt/homebrew/Caskroom/miniconda/base/envs/bm/lib/python3.11/asyncio/taskgroups.py", line 135, in __aexit__
  |     raise me from None
  | ExceptionGroup: unhandled errors in a TaskGroup (1 sub-exception)
  +-+---------------- 1 ----------------
    | Traceback (most recent call last):
    |   File "/Users/suzor/src/buttermilk/buttermilk/runner/orchestrator.py", line 87, in run_tasks
    |     async for agent_name, job_id, t